In [1]:
import json
import numpy as np
import faiss
from pathlib import Path

def main():
    # === Tự động định vị đường dẫn ===
    base_dir = Path.cwd().parent if Path.cwd().name == "scripts" else Path.cwd()
    data_dir = base_dir / "data"
    embeddings_file = data_dir / "embeddings.npy"
    chunks_file = data_dir / "chunked_docs.json"
    index_file = data_dir / "faiss_index.bin"
    meta_file = data_dir / "faiss_index.meta.json"

    # === Load dữ liệu ===
    embeddings = np.load(embeddings_file).astype('float32')
    with open(chunks_file, 'r', encoding='utf-8') as f:
        chunks = json.load(f)

    # === Kiểm tra khớp số lượng ===
    assert len(embeddings) == len(chunks), \
        f"Số lượng embeddings ({len(embeddings)}) không khớp với số chunk ({len(chunks)})"

    # === Tạo FAISS index (HNSW: hiệu quả cho truy vấn semantic) ===
    dim = embeddings.shape[1]
    index = faiss.IndexHNSWFlat(dim, 32)
    index.hnsw.efConstruction = 200
    index.add(embeddings)

    # === Lưu index và metadata ===
    faiss.write_index(index, str(index_file))
    with open(meta_file, 'w', encoding='utf-8') as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)

    print(f"✅ Đã tạo FAISS index: {index_file}")
    print(f"✅ Đã lưu metadata: {meta_file}")
    print(f"→ Tổng số vector: {len(embeddings)}, chiều vector: {dim}")

# === Chạy trực tiếp ===
main()


✅ Đã tạo FAISS index: d:\RAG\data\faiss_index.bin
✅ Đã lưu metadata: d:\RAG\data\faiss_index.meta.json
→ Tổng số vector: 7604, chiều vector: 384
